In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import calendar
from scipy.stats import chi2_contingency
from IPython.display import display
import os, re
project_folder =os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

In [5]:
data_general = pd.read_csv(f'{project_folder}/data/process/final_geo_coding.csv', skiprows=[1, 2]).reset_index()

In [5]:
# show the questions
pd.read_csv(f'{project_folder}/data/process/new_data.csv', skiprows=[2]).reset_index().iloc[0]

index                                                          0
StartDate                                             Start Date
EndDate                                                 End Date
Status                                             Response Type
IPAddress                                             IP Address
                                     ...                        
Q38            What is the highest level of education you hav...
Q39            What do you consider your main ethnic origin o...
Q39_10_TEXT    What do you consider your main ethnic origin o...
Q40            Where did you hear about this survey? - Select...
Q40_3_TEXT     Where did you hear about this survey? - Other ...
Name: 0, Length: 75, dtype: object

In [39]:
questions ={'Q2':'part','Q6':'day_time','Q7':'is_locked','Q8':'locked_type','Q9':'theft_location','Q10':'is_regi','Q11':'report_loc','Q12':'is_insured','Q13':'estimate_val','Q14':'is_electric','Q15':'bike_type','Q17':'is_university','Q19':'is_online','Q21':'is_police_assist','Q23':'recovery_loc','Q24':'con_recovery','Q34': 'birth', 'Q35': 'gender', 'Q36': 'income', 'Q38': 'education', 'Q39': 'ethnic_origin'}

In [118]:
def get_the_highest(table_updated, table_to_agg):
# This method aggregates small numbers of response types while maintaining 10% percentiles
    if table_to_agg[percentage].sum()<10:
        if len(table_to_agg)==1:
            # No aggregation is required
            table_updated = table_updated.append(table_to_agg)
        else:
            table_updated.loc['Aggregated Responses'] = round(table_to_agg.sum(), 1)
        return table_updated
    else:
        table_updated = table_updated.append(table_to_agg.iloc[0])
        return get_the_highest(table_updated, table_to_agg.iloc[1:])


In [95]:
def sub_aggregate_def(indices:list,j,sum)-> list:
    """
    add all the response types that together are less than 10%
    :param indices: list to updata
    :param j: next response types index to check
    :param sum: the current percentile sum
    :return:
    """
    temp_sum = sub_aggregate.iloc[j][percentage] +sum
    if temp_sum>10:
        return indices
    else:
        indices.append(j)
        if j+1 <len_sub_aggregate:
            indices = sub_aggregate_def(indices,j+1,temp_sum)
    return indices


In [225]:
new_tables = {}
percentage = 'percentage'
# Specific code for birth year


for pair in questions.items():
    # statistic by count (display also by percentage)
    # pair = ('Q39','ethnic_origin')
    question = pair[0]
    #  code for birth year has already conducted
    if question =='Q34':
        table_0 = DataFrame(data_general.groupby('Q34').count()['level_0'].sort_values(ascending=False).rename('count'))
        # convert year to decade.
        decade = 'decade'
        table_0[decade] = table_0.index.map(lambda x:int(x/10)*10)
        table = DataFrame(table_0.groupby(decade).sum()['count'].sort_values(ascending=False))
        table[percentage] = round(table['count']/table['count'].sum()*100,1)
        new_tables['Q34_birth'] = table.sort_values(by=decade)
        continue

    # work on specific question
    table_0 = DataFrame(data_general.groupby(question).count()['level_0'].sort_values(ascending=False).rename('count'))
    # Clean unnecessary data and white space
    table = table_0.set_index(table_0.index.map(lambda x: re.sub("\(.*?\)","",x)).str.replace('      ','').str.strip())

    table[percentage] = round(table['count']/table['count'].sum()*100,1)
    # Aggregate responses type with less than 10 responses
    table_aggregate = table[table[percentage]<10]
    table_new = table.copy()

    if len(table_aggregate)>1:
         # There are more than 1 values with less than 10% respondent
        table_new =table[table[percentage]>10]
        table_add = get_the_highest(DataFrame(columns=table_aggregate.columns),table_aggregate)
        # There are more than 1 values with less than 10% respondents that not part of aggregation process before
        if len(table_add)>2:
            sub_aggregate = table_add.iloc[:-1].sort_values(by=percentage)
            i =0
            len_sub_aggregate = len(sub_aggregate)
            while i < len_sub_aggregate:
                if i ==len_sub_aggregate -1:
                    indices = [i]
                else:
                    indices = sub_aggregate_def([i],i+1,sub_aggregate.iloc[i][percentage])
                # aggregate sll the responses type based on indices
                temp_df =sub_aggregate.iloc[indices]
                table_new.loc[' OR '.join(list(temp_df.index))] = temp_df.sum()
                i = indices[-1]+1
            table_new =table_new.append(table_add.iloc[-1])
        else:
            table_new = table_new.append(table_add)

    # add the new table into tables dictionary
    new_tables[f'{question}_{pair[1]}'] = table_new.sort_values(by=percentage,ascending=False)
    # break
with pd.ExcelWriter(f"{project_folder}/python/notebooks/output/output.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

C:\Users\Achituv\AppData\Local\Temp\ipykernel_32884\719687288.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_new = table_new.append(table_add)
C:\Users\Achituv\AppData\Local\Temp\ipykernel_32884\3031605931.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_updated.append(table_to_agg.iloc[0])
C:\Users\Achituv\AppData\Local\Temp\ipykernel_32884\3031605931.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_updated.append(table_to_agg.iloc[0])
C:\Users\Achituv\AppData\Local\Temp\ipykernel_32884\3031605931.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_updated = table_u

In [223]:
# United States VS Canada
new_tables = {}
percentage = 'percentage'
countries_names = list(data_general['country'].drop_duplicates())

countries_data = {countries_names[0]:data_general[data_general['country']==countries_names[0]],
                  countries_names[1]:data_general[data_general['country']==countries_names[1]]}

for pair in questions.items():
    by_countries_table = DataFrame(columns=countries_names)
    # statistic by count (display also by percentage)
    # pair = ('Q34', 'birth')
    question = pair[0]
    #  code for birth year has already conducted
    if question =='Q34':
        for item in countries_names:
            country_data= countries_data[item]
            # Specific code for birth year
            table_0 = DataFrame(country_data.groupby('Q34').count()['level_0'].sort_values(ascending=False).rename('count'))
            # convert year to decade.
            decade = 'decade'
            table_0[decade] = table_0.index.map(lambda x:int(x/10)*10)
            table = DataFrame(table_0.groupby(decade).sum()['count'].sort_values(ascending=False))
            by_countries_table[item] = round(table['count']/table['count'].sum()*100,1)

        by_countries_table.columns = by_countries_table.columns + '(%)'
        by_countries_table.fillna(0,inplace=True)
        new_tables[f'{question}_{pair[1]}'] = by_countries_table.sort_values(by=decade)
        continue
    for item in countries_names:
        country_data= countries_data[item]
        # work on specific question
        table_0 = DataFrame(country_data.groupby(question).count()['level_0'].sort_values(ascending=False).rename('count'))
        # Clean unnecessary data and white space
        table = table_0.set_index(table_0.index.map(lambda x: re.sub("\(.*?\)","",x)).str.replace('      ','').str.strip())
        by_countries_table[item] = round(table['count']/table['count'].sum()*100,1)
    by_countries_table.columns = by_countries_table.columns + '(%)'
    by_countries_table.fillna(0,inplace=True)
        # add the new table into tables dictionary
    new_tables[f'{question}_{pair[1]}'] = by_countries_table.sort_values(by='USA(%)',ascending=False)
    # break
with pd.ExcelWriter(f"{project_folder}/python/notebooks/output/usa_canada.xlsx") as writer:
    [item[1].to_excel(writer, sheet_name=item[0]) for item in new_tables.items()]

In [227]:
len(countries_data['CAN'])

338

In [228]:
len(countries_data['USA'])

1483

In [229]:
can = countries_data['CAN']
len(can[can['Q12']=='Yes'])/len(countries_data['CAN'])*100

0.0

In [230]:
can[can['Q12']=='Yes']

,level_0,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,StartDate,EndDate,Status,IPAddress,Progress,...,Q39_10_TEXT,Q40,Q40_3_TEXT,lat,lon,score,stolen_bikes_place,country,city,state


In [234]:
can['Q12']=='Yes'

0       False
3       False
9       False
11      False
14      False
        ...  
1779    False
1785    False
1788    False
1790    False
1819    False
Name: Q12, Length: 338, dtype: bool

In [235]:
can['Q12']

0       Don’t know/not sure 
3                       Yes 
9                       Yes 
11                      Yes 
14                      Yes 
                ...         
1779                     No 
1785                     No 
1788                    Yes 
1790                     No 
1819                     No 
Name: Q12, Length: 338, dtype: object